In [ ]:
from google.colab import drive
import os
import pandas as pd
import os
from PIL import Image
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from keras.utils.np_utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import os
import sys
from glob import glob
import cv2 
import time
import datetime
from tqdm import tqdm
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import torch
import os
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [ ]:
drive.mount('/content/drive')

In [ ]:
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = ""
!kaggle competitions download -c diabetic-retinopathy-detection -f train.zip.002
!kaggle competitions download -c diabetic-retinopathy-detection -f trainLabels.csv.zip
! unzip "trainLabels.csv.zip"
! unzip "train.zip.002.zip"
!apt install p7zip-full -y
!7z x train.zip.002
!ls train | wc -l #print number of images
!ls train

In [ ]:
base_image_dir = "/content/train"
retina_df = pd.read_csv("/content/trainLabels.csv")
retina_df['PatientId'] = retina_df['image'].map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir,
                                                         '{}.jpeg'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)
print(retina_df['exists'].sum(), 'images found of', retina_df.shape[0], 'total')
retina_df['eye'] = retina_df['image'].map(lambda x: 1 if x.split('_')[-1]=='left' else 0)
retina_df['level_cat'] = retina_df['level'].map(lambda x: to_categorical(x, 1+retina_df['level'].max()))

retina_df.dropna(inplace = True)
retina_df = retina_df[retina_df['exists']]
retina_df.sample(10)

In [ ]:
def balance_data(class_size,df):
    train_df = df.groupby(['level']).apply(lambda x: x.sample(class_size, replace = True)).reset_index(drop = True)
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    print('New Data Size:', train_df.shape[0], 'Old Size:', df.shape[0])
    train_df['level'].hist(figsize = (10, 5))
    return train_df
train_df = balance_data(2000,retina_df) # I will oversample such that all classes have the same number of images as the maximum
retina_df.pivot_table(index='level', aggfunc=len)
train_df.pivot_table(index='level', aggfunc=len)


In [ ]:
def get_label_and_image(path):
  name= path.split('/')[-1].split('.')[0]
  print(path,name)

  label=train_df["level_cat"].loc[train_df['image'] == name].values[0]
  print(label)
  image=Image.open(path)
  return image,label
image,label=get_label_and_image("/content/train/38791_left.jpeg")

In [ ]:
train_ds={"image":[],
            "label":[]}

paths=train_df["path"]
i=0
for path in paths:
  image,label=get_label_and_image(path)
  train_ds["image"].append(image)
  train_ds["label"].append(label)

In [ ]:
#data visualization
f, axarr = plt.subplots(2,2, figsize=(10, 10))

axarr[0,0].imshow(Image.open("/content/train/876_left.jpeg"))
axarr[0,0].set_title("level:"+str(retina_df["level"].loc[retina_df['image'] == "876_left"].values[0])) 

axarr[0,1].imshow(Image.open("/content/train/9203_left.jpeg"))
axarr[0,1].set_title("level:"+str(retina_df["level"].loc[retina_df['image'] == "9203_left"].values[0])) 

axarr[1,0].imshow(Image.open("/content/train/9977_right.jpeg"))
axarr[1,0].set_title("level:"+str(retina_df["level"].loc[retina_df['image'] == "9977_right"].values[0])) 

axarr[1,1].imshow(Image.open("/content/train/963_right.jpeg"))
axarr[1,1].set_title("level:"+str(retina_df["level"].loc[retina_df['image'] == "963_right"].values[0])) 

In [ ]:

ptm = InceptionResNetV2(
        input_shape = (100,100,3),
        weights = 'imagenet',
        include_top = False,
        pooling='max',
)
ptm.trainable = True
K = 5
x = Flatten()(ptm.output)
x = Dense(K, activation = 'softmax')(x)
model = Model(inputs = ptm.input , outputs = x)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [ ]:


# custom_early_stopping = EarlyStopping(
#     monitor='val_loss', 
#     patience=19, 
#     restore_best_weights=False)

# save_checkpoint_cb= callbacks.ModelCheckpoint(
#                       filepath='/content/drive/MyDrive/Biometrics Project/IRCP/IRCP', 
#                       save_freq='epoch', verbose=0,save_weights_only=True)

history=model.fit(
        x=train_images,
        y=train_labels,
        validation_data = (valid_images,valid_labels),
        epochs = 15,
        shuffle=True
)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'][:2]+history.history['accuracy'][4:14])
plt.plot(history.history['val_accuracy'][:2]+history.history['val_accuracy'][4:14])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'][:2]+history.history['loss'][4:14])
plt.plot(history.history['val_loss'][:2]+history.history['val_loss'][4:14])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save_weights("")

In [ ]:
model.evaluate(test_images,test_labels)